- pipeline
- tokenizer
- model
- datasets
- evaluate
- trainer

导入相关包
加载数据集
数据集划分
数据集预处理
创建模型
设置评估函数
配置训练参数
创建训练参数
模型训练、评估、预测
模型预测

显存占用分析
- 模型权重  4Bytes*模型参数量
- 优化器状态 8Bytes*模型参数量
- 梯度  4Bytes*模型参数量
- 前向激活值 取决于序列长度、隐层维度、batch大小等多个因素

显存占用优化策略分析
- gradient accmulation 梯度累加
- gradient checkpoints 正向传播的过程中边丢弃一些中间变量，在反向传播的时候再重新计算
- adafactor optimizer 不计算梯度的二阶正定矩阵，用了另外一种相似矩阵分解技术，加快了模型显存的占用
- freeze model 其实这是一种策略，而不是一种方法
- data length 输入数据的长度

| 优化策略                                      | 优化对象         | 显存占用 | 训练时间 |
|---------------------------------------------|-----------------|----------|----------|
| Baseline (BS 32, MaxLength 128)             | -               | 15.2G    | 64s      |
| + Gradient Accumulation (BS 1, GA 32)       | 前向激活值      |   7.4G       |    259s      |
| + Gradient Checkpoints (BS 1, GA 32)        | 前向激活值      |    7.2G      |     422s     |
| + Adafactor Optimizer (BS 1, GA 32)         | 优化器状态      |     5.0G     |     406s     |
| + Freeze Model (BS 1, GA 32)                | 前向激活值 / 梯度 |      3.4G    |      180s    |
| + Data Length (BS 1, GA 32, MaxLength 64)   | 前向激活值      |     3.3G     |      126s    |

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate
from transformers import DataCollatorWithPadding
from transformers import pipeline

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
datasets = dataset.train_test_split(test_size=0.1)

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)

model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")
acc_metric = evaluate.load("./metric_accuracy.py")
f1_metirc = evaluate.load("./metric_f1.py")

train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=2,   # 训练时的batch_size
                               gradient_accumulation_steps=32,  # *** 梯度累加 ***
                               gradient_checkpointing=True,     # *** 梯度检查点 ***
                               optim="adafactor",               # *** adafactor优化器 *** 
                               per_device_eval_batch_size=4,    # 验证时的batch_size
                               num_train_epochs=1,              # 训练轮数
                               logging_steps=10,                # log 打印的频率
                               eval_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.001,              # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型

# *** 参数冻结 *** 
for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model, 
                  args=train_args, 
                  tokenizer=tokenizer,
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

trainer.train()
trainer.evaluate(tokenized_datasets["test"])
trainer.predict(tokenized_datasets["test"])

In [ ]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)